In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# load the dataset (Train, Test)

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
print(train_data.head(3))
train_y = train_data['Survived']
train_X = train_data.drop(columns= ['Survived'])
print("Train data: ", train_X.head(3))
print("Train data: ", train_data.describe())

test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
# test data does not have survived
print("Test data: ", test_data.head(3))

Classification - What should I use?

In [ ]:
# exploring the patterns - male, female
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

# exploring the patterns - age
mean_age = train_data['Age'].mean()
print("mean age: ", mean_age)
above_mean_age = train_data.loc[train_data.Age >= 60]["Survived"]
rate_above_mean_age = sum(above_mean_age)/len(above_mean_age)
print("% of person age above 60 survived", rate_above_mean_age)
below_mean_age = train_data.loc[train_data.Age <= 19]["Survived"]
rate_below_mean_age = sum(below_mean_age)/len(below_mean_age)
print("% of person age below 19 survived", rate_below_mean_age)

In [ ]:
'''
# using logistic regression for classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
# from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, plot_roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import matplotlib.pyplot as plt

y = train_data["Survived"]
trainX = train_data.drop(['Name', 'Ticket', 'Cabin', 'Embarked', 'Survived', 'PassengerId', 'Parch', 'SibSp'], axis=1)
#trainX.replace('S', 1, inplace=True)
#trainX.replace('C', 0, inplace=True)

trainX.replace('male', 1, inplace=True)
trainX.replace('female', 0, inplace=True)
print(trainX.head())
model = LogisticRegression()
trainX_nr = StandardScaler().fit_transform(trainX)
# print(trainX_nr)

scaler = MinMaxScaler()
trainX_mm = scaler.fit_transform(trainX)
print(trainX_mm)
# trainX_mm.replace('nan', 1, inplace=True)
col_mean = np.nanmean(trainX_mm, axis=0)
print(col_mean)

#Find indices that you need to replace
inds = np.where(np.isnan(trainX_mm))

#Place column means in the indices. Align the arrays using take
trainX_mm[inds] = np.take(col_mean, inds[1])

model.fit(trainX_mm, y)

# train data
yPred_tr = model.predict(trainX_mm)
cm = confusion_matrix(y, yPred_tr)
print("train cm = ", cm)
print(classification_report(y, yPred_tr))
auc_tr = roc_auc_score(list(y), list(yPred_tr))
print("traing auc:", auc_tr)
fpr, tpr, thresholds = roc_curve(list(y), list(yPred_tr))
print("fpr: ", fpr)
print("tpr: ", tpr)
print("th: ", thresholds)
plot_roc_curve(model, trainX_mm, y)
plt.show()

logit_model=sm.OLS(y,trainX_mm)
result=logit_model.fit()
print(result.summary())

testX = test_data.drop(['Name', 'Ticket', 'Cabin', 'Embarked', 'PassengerId', 'Parch', 'SibSp'], axis=1)
testX.replace('male', 1, inplace=True)
testX.replace('female', 0, inplace=True)

testX_mm = scaler.fit_transform(testX)
print(testX_mm)
# trainX_mm.replace('nan', 1, inplace=True)
col_mean = np.nanmean(testX_mm, axis=0)
print(col_mean)

#Find indices that you need to replace
inds = np.where(np.isnan(testX_mm))

#Place column means in the indices. Align the arrays using take
testX_mm[inds] = np.take(col_mean, inds[1])

predictions = model.predict(testX_mm)
# print(predictions)

print("Score with logistic regression is 0.76")
'''

In [ ]:
'''
# using Decision Tree, Random Forest
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]


features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=3000, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

# result
print("score is 0.77")
'''

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

y = train_data["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = XGBClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f'%(np.mean(n_scores)))

model.fit(X,y)
predictions = model.predict(X_test)
print(predictions)

KeyboardInterrupt: 

In [ ]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")